## The method of stock prediction with various analize method (For practice)
- This is my python project for improving my skill
- For stock prediction, the proposed method uses various algorithms such as statistical analysis, probability model and neural networks
- The simple s&p500 data is used


## 다양한 분석 기법을 활용한 주가 예측 방법 (연습) 
- 파이썬을 활용하여 다양한 분석 기법을 활용한 주가 예측 방법 구현 수행 (습작용)
- 그냥 널리 알려져 있는 통계적 분석, 확률모델, 인공 신경망 등등 여러 방법 활용
- 데이터 역시 단순한 종가 등 수학적 정보만 활용 
- 어떤 분석 방법이 예측력이 좋은지, 어떻게 하면 예측력을 높일 수 있는지 등